# Code and Documentation
### Part 3 - Models
____
#### GroupD: Nicole Guobadia, Dhruvi Kaswala, Arashpreet Singh Pandher, Karen Angella Edy Setiawan

Note: make sure to shut down all other kernels

In [1]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install()
BiocManager::install("randomForest")
library (randomForest)

Bioconductor version '3.10' is out-of-date; the current release version '3.16'
  is available with R version '4.2'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.10 (BiocManager 1.30.20), R 3.6.1 (2019-07-05)

Old packages: 'askpass', 'backports', 'BH', 'boot', 'brio', 'broom', 'callr',
  'caret', 'class', 'cli', 'clipr', 'cluster', 'codetools', 'collections',
  'colorspace', 'commonmark', 'crayon', 'curl', 'data.table', 'DBI', 'dbplyr',
  'dichromat', 'diffobj', 'digest', 'dplyr', 'e1071', 'evaluate', 'fansi',
  'forcats', 'foreach', 'formatR', 'fs', 'generics', 'ggplot2', 'glmnet',
  'glue', 'gower', 'gtable', 'gtools', 'haven', 'hexbin', 'highr', 'hms',
  'htmltools', 'htmlwidgets', 'httpuv', 'httr', 'ipred', 'IRdisplay',
  'IRkernel', 'iterators', 'jsonlite', 'KernSmooth', 'k

In [2]:
install.packages('e1071', dependencies=TRUE)
library('e1071')


  There is a binary version available but the source version is later:
      binary source needs_compilation
e1071  1.7-6 1.7-13              TRUE

  Binaries will be installed
package 'e1071' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\nicol\AppData\Local\Temp\RtmpknDpno\downloaded_packages


Warning message:
"package 'e1071' was built under R version 3.6.3"


In [3]:
set.seed(23)

In [4]:
# load train, test, and gene signature data into notebook
load("data.RData")
load("gene.RData")
ls()

[1] "genesig"       "test_dataset"  "train_dataset"

In [5]:
train_dataset[1:5, 1:5]

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD
GSM1030962_10030.CEL,8.27154,5.02307,7.06640,8.24990,5.75432
GSM1030963_10110.CEL,8.49112,3.95575,6.41582,8.42649,6.77953
GSM1030964_10111.CEL,8.34082,4.68666,6.14716,8.09809,6.35168
GSM1030966_1032.CEL,7.75355,4.97396,6.35464,8.00680,6.79648
GSM1030969_1446.CEL,8.40749,4.55103,6.63393,8.56563,6.34759


In [6]:
test_dataset[1:5, 1:5]

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD
GSM1030965_10138.CEL,8.24190,5.78429,6.65615,8.15517,4.97367
GSM1030967_1113.CEL,7.95457,5.75343,6.89049,7.95516,5.49744
GSM1030968_1132.CEL,7.99231,4.21497,6.28378,8.13866,7.27109
GSM1030979_2120_opnieuw.CEL,8.08617,6.31566,7.07061,8.51588,6.85328
GSM1030982_2252.CEL,7.93179,5.32106,6.42760,8.52095,6.37300


In [7]:
genesig

[1]   283   407   619   680  1738  1780  1885  2025  2303  2304  2374  2651
[13]  2656  2855  3034  3275  3325  3524  3541  3666  3733  3764  3844  3893
[25]  4292  4806  5047  5278  5482  5604  5777  5835  6268  7159  7458  7652
[37]  8075  8129  8457  8493  8602  8737  8827  8887  9338  9789  9794  9877
[49] 10136 10443 10788 10818 11364 11941 11998 12146

##### We train linear SVM model and gives the confusion matrix and accuracy:

In [8]:
# train data with linear svm
train <- train_dataset[, genesig]
train$label <- train_dataset$label
classifier.svm <- svm(label ~ ., data = train, type = 'C-classification', kernel = 'linear')

In [9]:
# predict using test data
pred.svm <- predict(classifier.svm, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.svm, test_dataset$label)
conf_matrix

        
pred.svm   0   1
       0 507   4
       1   7 547

In [10]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9896714

##### We train logistic regression model and gives the confusion matrix and accuracy:

In [11]:
# train data with log reg
classifier.logreg <- glm(label ~ ., data = train,family = binomial)

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"


In [12]:
# predict using test data
pred.logreg <- predict(classifier.logreg, test_dataset[,1:length(test_dataset)-1], type = "response")
pred.logreg <- round(pred.logreg)
conf_matrix <- table(pred.logreg, test_dataset$label)
conf_matrix

           
pred.logreg   0   1
          0 508   5
          1   6 546

In [13]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9896714

##### We train random forest model and gives the confusion matrix and accuracy:

In [14]:
# randomforest classifier
classifier.tree <- randomForest(as.factor(train$label) ~ ., data=train)

In [15]:
pred.tree <- predict (classifier.tree, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.tree, test_dataset$label)
conf_matrix

         
pred.tree   0   1
        0 511  10
        1   3 541

In [16]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9877934

##### Build ensemble with majority voting:

In [17]:
#The majority vote
# reference https://www.analyticsvidhya.com/blog/2017/02/introduction-to-ensembling-along-with-implementation-in-r/
testSet <- as.factor(ifelse(pred.tree == '1' & pred.logreg == '1', '1', ifelse(pred.tree
                                                                        == '1' & pred.svm == '1','1', ifelse(pred.logreg == '1' & pred.svm == '1', '1', '0'))))

In [18]:
conf_matrix <- table(testSet, test_dataset$label)
conf_matrix

       
testSet   0   1
      0 509   5
      1   5 546

In [19]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9906103

### Alternate gene signature comparison from other team (Genome Seeker)

##### We train all the models again with new gene signatures:

In [20]:
alt.genesig <- colnames(read.table("alternate_genesig.txt", sep=",", header=T))
alt.genesig

[1] "IGHA2"     "SOCS2"     "IGKC"      "KIT"       "PRAME"     "IGLJ2"    
 [7] "PPBP"      "ALDH1A1"   "PDGFC"     "CLC"       "IGF2BP2"   "EPS8"     
[13] "HOXB6"     "PF4"       "G0S2"      "HOXA9"     "SECISBP2L" "DEFA1"    
[19] "HDC"       "PTH2R"     "FGF13"     "CST3"      "RORA"      "CXCL3"    
[25] "DPY19L1P1" "PAX5"      "ADAM28"    "DNM1"      "FPR1"      "NR4A3"    
[31] "TFPI"      "TOP1"      "LEF1"      "TST"       "SLCO3A1"   "CTSW"     
[37] "PRSS21"    "TIMP1"     "CYP4F3"    "TMEM243"   "ALOX12"    "ABL1"     
[43] "PROS1"     "LRMP"      "DISC1"     "MYH10"     "RNF216"    "FAN1"     
[49] "HOMER3"

In [21]:
alt.genesig.ind <- c(1:49)
for (j in 1:49){
    alt.genesig.ind[j] <- which(colnames(train_dataset) == alt.genesig[j])
}
alt.genesig.ind[1:5]

[1]  8827  1738 12198  2855  2212

In [22]:
# Train models
train <- train_dataset[, alt.genesig.ind]
train$label <- train_dataset$label
classifier.svm <- svm(label ~ ., data = train, type = 'C-classification', kernel = 'linear')
classifier.logreg <- glm(label ~ ., data = train,family = binomial)
classifier.tree <- randomForest(as.factor(train$label) ~ ., data=train)

In [23]:
# SVM prediction
pred.alt.svm <- predict(classifier.svm, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.alt.svm, test_dataset$label)
conf_matrix

            
pred.alt.svm   0   1
           0 507  14
           1   7 537

In [24]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9802817

In [25]:
# Logistic regression prediction
pred.alt.logreg <- predict(classifier.logreg, test_dataset[,1:length(test_dataset)-1], type = "response")
pred.alt.logreg <- round(pred.alt.logreg)
conf_matrix <- table(pred.alt.logreg, test_dataset$label)
conf_matrix

               
pred.alt.logreg   0   1
              0 508  15
              1   6 536

In [26]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9802817

In [27]:
# Random forest prediction
pred.alt.tree <- predict (classifier.tree, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.alt.tree, test_dataset$label)
conf_matrix

             
pred.alt.tree   0   1
            0 509  14
            1   5 537

In [28]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9821596

##### Build ensemble with majority voting:

In [29]:
#The majority vote
# reference https://www.analyticsvidhya.com/blog/2017/02/introduction-to-ensembling-along-with-implementation-in-r/
testSet <- as.factor(ifelse(pred.alt.tree == '1' & pred.alt.logreg == '1', '1', ifelse(pred.alt.tree
                                                                        == '1' & pred.alt.svm == '1','1', ifelse(pred.alt.logreg == '1' & pred.alt.svm == '1', '1', '0'))))

In [30]:
conf_matrix <- table(testSet, test_dataset$label)
conf_matrix

       
testSet   0   1
      0 507  13
      1   7 538

In [31]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9812207

## Cross validation on dataset 1

In [32]:
load("AML_datasets.RData")

### Preprocessing of dataset

In [33]:
# gene expression data
dim(data.1)
data.1[1:5, 1:8]

[1] 12708  2500

,AHW_ALL_BA_025919_13_A_PZ.CEL,AHW_ALL_BA_032751_71_A_PZ.CEL,AHW_ALL_BN_022495_7_A_PZ.CEL,AHW_ALL_MLL_031351_13_A_PZ.CEL,AHW_ALL_MLL_036187_80_A_PZ.CEL,AHW_ALL_Ph_030031_32_A_PZ.CEL,AHW_ALL_Ph_032683_15_A_PZ.CEL,AHW_ALL_Ph_035655_63_A_PZ.CEL
PAX8,9.221573,8.524435,8.606060,8.841501,8.882826,9.011544,8.975590,8.940409
CCL5,7.990144,6.648112,6.382602,6.469583,8.332415,6.830385,8.753525,6.370298
MMP14,8.391732,8.183514,7.909023,8.105744,7.820992,7.859977,8.128827,7.956449
DTX2P1-UPK3BP1-PMS2P11,9.280415,8.798847,8.948909,8.941517,8.725958,8.975915,9.130401,8.997386
BAD,5.579486,5.934621,5.646312,5.899868,5.651731,5.798776,5.849792,6.002240


In [34]:
# subset of data containing only AML and ALL samples
leukemia <- c('AML', 'ALL')
subset.info <- subset(info.1, Disease %in% leukemia)

In [35]:
# phenotype data (AML and ALL only)
dim(subset.info)
subset.info[1:5,]

[1] 1942    8

,Dataset,GSE,Condition,Disease,Tissue,FAB,Filename,FAB_all
AHW_ALL_BA_025919_13_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_BA_025919_13_A_PZ.CEL,unknown
AHW_ALL_BA_032751_71_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_BA_032751_71_A_PZ.CEL,unknown
AHW_ALL_BN_022495_7_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_BN_022495_7_A_PZ.CEL,unknown
AHW_ALL_MLL_031351_13_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_MLL_031351_13_A_PZ.CEL,unknown
AHW_ALL_MLL_036187_80_A_PZ.CEL,1,Haferlach,CONTROL,ALL,BM or PBMC,unknown,AHW_ALL_MLL_036187_80_A_PZ.CEL,unknown


In [36]:
# gene expression data (AML and ALL only)
subset.data.1 <- data.1[,rownames(subset.info)]
dim(subset.data.1)
subset.data.1[1:10,1:10]

[1] 12708  1942

,AHW_ALL_BA_025919_13_A_PZ.CEL,AHW_ALL_BA_032751_71_A_PZ.CEL,AHW_ALL_BN_022495_7_A_PZ.CEL,AHW_ALL_MLL_031351_13_A_PZ.CEL,AHW_ALL_MLL_036187_80_A_PZ.CEL,AHW_ALL_Ph_030031_32_A_PZ.CEL,AHW_ALL_Ph_032683_15_A_PZ.CEL,AHW_ALL_Ph_035655_63_A_PZ.CEL,AHW_ALL_TA_025336_26_A_PZ.CEL,AHW_ALL_TA_032725_16_A_PZ.CEL
PAX8,9.221573,8.524435,8.606060,8.841501,8.882826,9.011544,8.975590,8.940409,9.342263,8.966156
CCL5,7.990144,6.648112,6.382602,6.469583,8.332415,6.830385,8.753525,6.370298,6.971186,7.302477
MMP14,8.391732,8.183514,7.909023,8.105744,7.820992,7.859977,8.128827,7.956449,8.285775,8.335117
DTX2P1-UPK3BP1-PMS2P11,9.280415,8.798847,8.948909,8.941517,8.725958,8.975915,9.130401,8.997386,9.294638,9.094800
BAD,5.579486,5.934621,5.646312,5.899868,5.651731,5.798776,5.849792,6.002240,5.642667,5.768286
PRPF8,9.587992,10.605028,10.367226,10.493261,10.684440,9.850415,10.068460,10.259652,10.168444,9.143968
CAPNS1,10.143771,10.613034,9.836321,10.701126,10.524360,10.115055,10.252378,10.722423,9.809753,9.503368
RPL35,11.364570,11.664621,12.081325,12.029874,11.856132,11.909563,11.671361,12.208231,10.956313,11.262788
EIF4G2,10.956290,11.632159,11.690962,11.328697,11.388613,11.197291,11.070642,11.471852,11.881013,11.051533
EIF3D,10.026995,11.412836,10.966108,11.176077,11.117741,10.518100,11.040760,11.182870,10.474084,10.038692


In [37]:
subset.data.1 <- as.data.frame(t(subset.data.1))
dim(subset.data.1)
subset.data.1[1:10,1:10]

[1]  1942 12708

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD,PRPF8,CAPNS1,RPL35,EIF4G2,EIF3D
AHW_ALL_BA_025919_13_A_PZ.CEL,9.221573,7.990144,8.391732,9.280415,5.579486,9.587992,10.143771,11.36457,10.95629,10.02699
AHW_ALL_BA_032751_71_A_PZ.CEL,8.524435,6.648112,8.183514,8.798847,5.934621,10.605028,10.613034,11.66462,11.63216,11.41284
AHW_ALL_BN_022495_7_A_PZ.CEL,8.606060,6.382602,7.909023,8.948909,5.646312,10.367226,9.836321,12.08132,11.69096,10.96611
AHW_ALL_MLL_031351_13_A_PZ.CEL,8.841501,6.469583,8.105744,8.941517,5.899868,10.493261,10.701126,12.02987,11.32870,11.17608
AHW_ALL_MLL_036187_80_A_PZ.CEL,8.882826,8.332415,7.820992,8.725958,5.651731,10.684440,10.524360,11.85613,11.38861,11.11774
AHW_ALL_Ph_030031_32_A_PZ.CEL,9.011544,6.830385,7.859977,8.975915,5.798776,9.850415,10.115055,11.90956,11.19729,10.51810
AHW_ALL_Ph_032683_15_A_PZ.CEL,8.975590,8.753525,8.128827,9.130401,5.849792,10.068460,10.252378,11.67136,11.07064,11.04076
AHW_ALL_Ph_035655_63_A_PZ.CEL,8.940409,6.370298,7.956449,8.997386,6.002240,10.259652,10.722423,12.20823,11.47185,11.18287
AHW_ALL_TA_025336_26_A_PZ.CEL,9.342263,6.971186,8.285775,9.294638,5.642667,10.168444,9.809753,10.95631,11.88101,10.47408
AHW_ALL_TA_032725_16_A_PZ.CEL,8.966156,7.302477,8.335117,9.094800,5.768286,9.143968,9.503368,11.26279,11.05153,10.03869


In [38]:
# add class label
subset.data.1$label <- with(subset.info, ifelse(Disease=='AML', 1, 0))
test.dataset.1 <- subset.data.1
test.dataset.1$label[1:10]
dim(test.dataset.1)
length(which(test.dataset.1$label == 1))
length(which(test.dataset.1$label == 0))

[1] 0 0 0 0 0 0 0 0 0 0

[1]  1942 12709

[1] 986

[1] 956

### Linear SVM cross validation

In [39]:
# predict using test data
pred.svm <- predict(classifier.svm, test.dataset.1[,1:length(test.dataset.1)-1], decision.values = TRUE)
conf_matrix <- table(pred.svm, test.dataset.1$label)
conf_matrix

        
pred.svm   0   1
       0 924   9
       1  32 977

In [40]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9788877

### Logistic regression cross validation

In [41]:
# predict using test data
pred.logreg <- predict(classifier.logreg, test.dataset.1[,1:length(test.dataset.1)-1], type = "response")
pred.logreg <- round(pred.logreg)
conf_matrix <- table(pred.logreg, test.dataset.1$label)
conf_matrix

           
pred.logreg   0   1
          0 933   8
          1  23 978

In [42]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9840371

### Decision tree cross validation

In [43]:
pred.tree <- predict (classifier.tree, test.dataset.1[,1:length(test.dataset.1)-1], decision.values = TRUE)
conf_matrix <- table(pred.tree, test.dataset.1$label)
conf_matrix

         
pred.tree   0   1
        0 928   5
        1  28 981

In [44]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9830072

### Ensemble cross validation

In [45]:
testSet <- as.factor(ifelse(pred.tree == '1' & pred.logreg == '1', '1', ifelse(pred.tree
                                                                        == '1' & pred.svm == '1','1', ifelse(pred.logreg == '1' & pred.svm == '1', '1', '0'))))
conf_matrix <- table(testSet, test.dataset.1$label)
conf_matrix

       
testSet   0   1
      0 929   8
      1  27 978

In [46]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9819773

## Cross validation on dataset 3
Note that dataset 3 has a very big class imbalance. This will be a nice set to test on as our model was trained on ~balanced data, so theoretically the imbalance should not effect the AML vs ALL classification to much (which it seems not to due to the ensemble implementation).

### Preprocessing of dataset

In [47]:
# gene expression data
dim(data.3)
data.3[1:5, 1:8]

[1] 12708  1181

,GSM1113367,GSM1113368,GSM1113369,GSM1113370,GSM1113371,GSM1113372,GSM1113373,GSM1113374
PAX8,147.8479,207.7174,146.1076,143.4009,124.5015,168.9003,154.3915,181.0532
CCL5,7007.8725,5185.2789,7372.0930,7640.6021,11859.1891,7915.5822,8649.7113,6770.8353
MMP14,141.6314,405.2492,169.6150,216.6392,156.6613,156.3427,126.2342,187.2506
DTX2P1-UPK3BP1-PMS2P11,1022.5923,878.7947,930.7014,966.7274,974.5085,908.5628,1056.4850,1018.7891
BAD,262.8271,316.5579,346.3744,236.2970,361.4526,272.7587,261.6820,233.0122


In [48]:
# subset of data containing only AML and ALL samples
leukemia <- c('AML', 'ALL')
subset.info <- subset(info.3, Disease %in% leukemia)

In [49]:
dim(subset.info)
subset.info[1:5,]

[1] 520   7

,Dataset,GSE,Condition,Disease,Tissue,FAB,Filename
GSM1202460,3,Simon (GSE49601),CONTROL,ALL,PBMC,unknown,GSM1202460
GSM1202461,3,Simon (GSE49601),CONTROL,ALL,BM,unknown,GSM1202461
GSM1202462,3,Simon (GSE49601),CONTROL,ALL,PBMC,unknown,GSM1202462
GSM1202463,3,Simon (GSE49601),CONTROL,ALL,BM,unknown,GSM1202463
GSM1202464,3,Simon (GSE49601),CONTROL,ALL,BM,unknown,GSM1202464


In [50]:
subset.data.3 <- data.3[,rownames(subset.info)]
dim(subset.data.3)
subset.data.3[1:10,1:10]

[1] 12708   520

,GSM1202460,GSM1202461,GSM1202462,GSM1202463,GSM1202464,GSM1202465,GSM1202466,GSM1202467,GSM1202468,GSM1202469
PAX8,429.14191,342.4850,346.0292,273.10056,219.73395,10.36487,359.15587,412.0841,210.9968,2.330412
CCL5,679.59668,770.7853,178.6169,105.78929,815.02943,260.46515,189.92748,416.3366,104.7086,232.540562
MMP14,46.22431,184.8808,119.2873,51.84463,43.38935,93.64472,60.19456,111.4081,79.3114,177.514014
DTX2P1-UPK3BP1-PMS2P11,937.95926,2558.8866,1148.2014,857.68102,951.90475,965.23117,910.78529,982.7911,1020.8992,1777.440146
BAD,1608.76106,1188.0667,697.1790,534.30104,349.51153,500.22082,495.15526,655.8174,827.3081,362.403278
PRPF8,27720.63275,24220.2543,18303.9556,13737.03256,13659.25131,14420.29483,12919.00294,23050.4315,17090.0000,13946.715964
CAPNS1,12820.85508,12869.8397,7988.8335,4964.47925,4370.81172,5288.43875,4663.29429,5430.1533,6431.5020,3936.225143
RPL35,13999.22541,13935.4712,4892.0820,21213.43660,7804.72465,13878.59571,7996.20608,13925.1113,5370.0660,13112.183915
EIF4G2,30957.22408,18645.1055,28528.6323,20873.14886,38471.19861,28300.52006,33028.51305,65284.8969,38138.4645,18681.880544
EIF3D,14112.16550,10982.3758,8830.4264,7660.99569,5333.29482,9834.57141,6256.77562,10280.4448,5115.3989,11652.643396


In [51]:
subset.data.3 <- as.data.frame(t(subset.data.3))
dim(subset.data.3)
subset.data.3[1:10,1:10]

[1]   520 12708

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD,PRPF8,CAPNS1,RPL35,EIF4G2,EIF3D
GSM1202460,429.141908,679.5967,46.22431,937.9593,1608.7611,27720.63,12820.855,13999.225,30957.22,14112.165
GSM1202461,342.484974,770.7853,184.88082,2558.8866,1188.0667,24220.25,12869.840,13935.471,18645.11,10982.376
GSM1202462,346.029244,178.6169,119.28728,1148.2014,697.1790,18303.96,7988.833,4892.082,28528.63,8830.426
GSM1202463,273.100560,105.7893,51.84463,857.6810,534.3010,13737.03,4964.479,21213.437,20873.15,7660.996
GSM1202464,219.733952,815.0294,43.38935,951.9048,349.5115,13659.25,4370.812,7804.725,38471.20,5333.295
GSM1202465,10.364874,260.4651,93.64472,965.2312,500.2208,14420.29,5288.439,13878.596,28300.52,9834.571
GSM1202466,359.155866,189.9275,60.19456,910.7853,495.1553,12919.00,4663.294,7996.206,33028.51,6256.776
GSM1202467,412.084139,416.3366,111.40809,982.7911,655.8174,23050.43,5430.153,13925.111,65284.90,10280.445
GSM1202468,210.996819,104.7086,79.31140,1020.8992,827.3081,17090.00,6431.502,5370.066,38138.46,5115.399
GSM1202469,2.330412,232.5406,177.51401,1777.4401,362.4033,13946.72,3936.225,13112.184,18681.88,11652.643


In [52]:
# add class label
subset.data.3$label <- with(subset.info, ifelse(Disease=='AML', 1, 0))
test.dataset.3 <- subset.data.3
test.dataset.3$label[1:10]
dim(test.dataset.3)
length(which(test.dataset.3$label == 1))
length(which(test.dataset.3$label == 0))

[1] 0 0 0 0 0 0 0 0 0 0

[1]   520 12709

[1] 508

[1] 12

### Linear SVM cross validation

In [53]:
pred.svm <- predict(classifier.svm, test.dataset.3[,1:length(test.dataset.3)-1], decision.values = TRUE)
conf_matrix <- table(pred.svm, test.dataset.3$label)
conf_matrix

        
pred.svm   0   1
       0   8   7
       1   4 501

In [54]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9788462

### Logistic regression cross validation

In [55]:
pred.logreg <- predict(classifier.logreg, test.dataset.3[,1:length(test.dataset.3)-1], type = "response")
pred.logreg <- round(pred.logreg)
conf_matrix <- table(pred.logreg, test.dataset.3$label)
conf_matrix

           
pred.logreg   0   1
          0   9   8
          1   3 500

In [56]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9788462

### Decision tree cross validation

In [57]:
# note the poor-er performance here; should take note in the paper...
pred.tree <- predict (classifier.tree, test.dataset.3[,1:length(test.dataset.3)-1], decision.values = TRUE)
conf_matrix <- table(pred.tree, test.dataset.3$label)
conf_matrix

         
pred.tree   0   1
        0  11 154
        1   1 354

In [58]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.7019231

### Ensemble cross validation

In [59]:
testSet <- as.factor(ifelse(pred.tree == '1' & pred.logreg == '1', '1', ifelse(pred.tree
                                                                        == '1' & pred.svm == '1','1', ifelse(pred.logreg == '1' & pred.svm == '1', '1', '0'))))
conf_matrix <- table(testSet, test.dataset.3$label)
conf_matrix

       
testSet   0   1
      0   9   8
      1   3 500

In [60]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9788462